### This is example of communication among participants in **DIDCOMM** manner
We have 3 members: **Alice**, **Bob**, **Carl**. 

For demo purposes we maintain Wallets in the cloud<br/>
You can check it by visiting [https://agents.socialsirius.com/](https://agents.socialsirius.com/) <br/>
Login: **didcomm** Password: **didcomm**


In [34]:
import sirius_sdk

# Init connection to Wallet that maintained in the Cloud
# Off cource developer have alternative - he may maintain Wallet and secrets on other way (KERI library) 
# thanks to Sirius-SDK is friendly to Legacy and flexible due to dependency injection architecture 

carl_cfg = sirius_sdk.Config().setup_cloud(
    server_uri='https://agents.socialsirius.com',
    credentials='s7RxiBlpeNq8k8hrx4vlgjb8XFnGQGxTjIQgk74LgYSYAS4TuR1kZZxJg5MS6b+rdefbzDIW5SbyYbkI+UYP/Q==',
    p2p={
        "their_verkey": "HCNPicRwYQHhdQ7Z5nNQ8kaAQkLeetmLctfyZ5GVxDy9",
        "my_keys": [
          "DQTDLvkSAjGYmKG5a9EvgvUwJzKXQfQuqxwgYHz6H9jE",
          "2dUPJxWVbNU1VhsUFAkVNLeq3kG7TghBLum8atfy6C3QAkG4GAYxAWEU4NMy3Qi9BGVdehP7UATaDdhw8g79A2jJ"
        ]
   }
)

### Ok, we initialized our environment, Let's establish connections with Alice (in our demo Alice is Inviter)
Put here invitation that **Alice** was generated earlier...

In [35]:
import json
import logging
import asyncio
from typing import Optional

import ipywidgets as widgets

# 
logging.getLogger().setLevel(logging.CRITICAL)

# Variables to store Peer-To-Peer metadata to communicate
ALICE: Optional[sirius_sdk.Pairwise] = None
# Seed to restore self identity
CARL_SEED = '0000000000000000000000000000CARL'


async def accept_invitation(url: str):
    try:
        async with sirius_sdk.context(carl_cfg):
            invitation = sirius_sdk.aries_rfc.Invitation.from_url(url)
            output.append_stdout('\nInvitation JSON:\n')
            output.append_stdout(json.dumps(invitation, indent=2, sort_keys=True))
            
            output.append_stdout('\n !!!!!!!!! Start Aries-RFC-0160 protocol !!!!!!')
            did, verkey = await sirius_sdk.DID.create_and_store_my_did(seed=CARL_SEED)
            endpoint = [e for e in await sirius_sdk.endpoints() if e.routing_keys == []][0]
            output.append_stdout(f'\nMy DID: {did}  My Verkey: {verkey}  My Endpoint: {endpoint.address}\n')
            
            rfc_0160 = sirius_sdk.aries_rfc.Invitee(
                me=sirius_sdk.Pairwise.Me(did, verkey),
                my_endpoint=endpoint,
                time_to_live=15  # 15 sec limit
            )
            success, p2p = await rfc_0160.create_connection(
                invitation=invitation,
                my_label='Carl'
            )
            if success:
                ALICE = p2p
                output.append_stdout('_'*128 + '\n')
                output.append_stdout('\n!!! Connection with Alice was established !!!!')
                output.append_stdout('\n' + '_'*128)
            else:
                output.append_stderr(rfc_0160.problem_report.explain)
    except Exception as e:
        output.append_stderr(repr(e))


output = widgets.Output()
txt = widgets.Text()
button = widgets.Button(description="Accept Invitation", button_style='primary')
def on_button_clicked(b):
    output.clear_output()
    asyncio.ensure_future(accept_invitation(txt.value))
button.on_click(on_button_clicked)
hboxes = [txt, button]

display(widgets.HBox(hboxes), output)

Output()